In [47]:
from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [48]:
gc = Gcrf()

In [49]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [50]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [51]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [52]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [53]:
def PivotAroundEdge(e, P, H=[]):
    print(e)
    p = P[-1]
    area2 = gc.square_area(e.p1, e.p2, p)
    for p_ in range(len(P)-1):
        volume = gc.signed_volume(e.p1, e.p2, p, P[p_])
        print("P",p)
        print("P_", P[p_])
        print("VOLUME", volume)
        face_no_hull = checa_face(H, Face(e, Edge(e.p1, P[p_]), Edge(P[p_], e.p2)))
        print("FACE NO HULL", face_no_hull)
        if volume < 0 and not face_no_hull:
            p = P[p_]
        elif volume == 0:
            area2_ = gc.square_area(e.p1, e.p2, P[p_])
            print(area2)
            print(area2_)
            if area2_ > area2 and not face_no_hull:
                p = P[p_]
                area2 = area2_
    print("PRÓXIMO PONTO", p)
    return p

In [54]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [55]:
def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = p

    for r in P:
        if (q.z == r.z) and (q.y == r.y) and (q.x < r.x):
            q = r
    
    if (q == p):
        q = Point(p.x + 1, p.y, p.z)
        print("Chamando PIVOT no Find Edge on Hull")
        q = PivotAroundEdge(Edge(p, q), P)

    return Edge(p, q)

In [56]:
def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

In [57]:
def NotProcessed(E, e):
    print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [58]:
def GiftWrap(P):
    t = FindTriangleOnHull(P)
    print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = [t.e1]

    while not len(Q) == 0:
        e = Q.pop()
#         print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
#             print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P, H)
#             print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
#             print("{} entrou no Hull".format(t))
            Q.extend([t.e1, t.e2, t.e3])
#             print("Q agora possui as arestas:")
#             for q in Q: print(q)
            Processed.append(e)
#             print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
#             for aresta in Processed: print(aresta)
        else: print("Aresta JÁ PROCESSADA")
    return H

In [59]:
p1 = Point(-3, 0, 0)
p2 = Point(0, -3, 0)
p3 = Point(0, 2, 0)
p4 = Point(0, 0, 3)
p5 = Point(0, 0, -4)
p6 = Point(4, 0, 0)
p7 = Point(0, 0, 0)

P = [p1, p2, p3, p4, p5, p6, p7]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

Chamando PIVOT no Find Edge on Hull
[(0, -3, 0) -> (1, -3, 0)]
P (0, 0, 0)
P_ (-3, 0, 0)
VOLUME 0.0
FACE NO HULL False
4.5
4.5
P (0, 0, 0)
P_ (0, -3, 0)
VOLUME 0.0
FACE NO HULL False
4.5
0.0
P (0, 0, 0)
P_ (0, 2, 0)
VOLUME 0.0
FACE NO HULL False
4.5
12.5
P (0, 2, 0)
P_ (0, 0, 3)
VOLUME 2.5
FACE NO HULL False
P (0, 2, 0)
P_ (0, 0, -4)
VOLUME -3.3333333333333335
FACE NO HULL False
P (0, 0, -4)
P_ (4, 0, 0)
VOLUME 2.0
FACE NO HULL False
PRÓXIMO PONTO (0, 0, -4)
[(0, -3, 0) -> (0, 0, -4)]
P (0, 0, 0)
P_ (-3, 0, 0)
VOLUME -6.0
FACE NO HULL False
P (-3, 0, 0)
P_ (0, -3, 0)
VOLUME 0.0
FACE NO HULL False
72.0
0.0
P (-3, 0, 0)
P_ (0, 2, 0)
VOLUME 10.0
FACE NO HULL False
P (-3, 0, 0)
P_ (0, 0, 3)
VOLUME 10.5
FACE NO HULL False
P (-3, 0, 0)
P_ (0, 0, -4)
VOLUME 0.0
FACE NO HULL False
72.0
0.0
P (-3, 0, 0)
P_ (4, 0, 0)
VOLUME 14.0
FACE NO HULL False
PRÓXIMO PONTO (-3, 0, 0)
Triangulo encontrado: {[(0, -3, 0) -> (0, 0, -4)], [(0, 0, -4) -> (-3, 0, 0)], [(-3, 0, 0) -> (0, -3, 0)]}
Aresta no Not Proc

P_ (4, 0, 0)
VOLUME 0.0
FACE NO HULL True
72.0
0.0
PRÓXIMO PONTO (0, 2, 0)
Nova face: {[(4, 0, 0) -> (0, 0, 3)], [(0, 0, 3) -> (0, 2, 0)], [(0, 2, 0) -> (4, 0, 0)]}
Aresta no Not Processed [(0, 2, 0) -> (4, 0, 0)]
[(0, 2, 0) -> (4, 0, 0)]
P (0, 0, 0)
P_ (-3, 0, 0)
VOLUME 0.0
FACE NO HULL False
32.0
98.0
P (-3, 0, 0)
P_ (0, -3, 0)
VOLUME 0.0
FACE NO HULL False
98.0
200.0
P (0, -3, 0)
P_ (0, 2, 0)
VOLUME 0.0
FACE NO HULL True
200.0
0.0
P (0, -3, 0)
P_ (0, 0, 3)
VOLUME -10.0
FACE NO HULL False
P (0, 0, 3)
P_ (0, 0, -4)
VOLUME 9.333333333333334
FACE NO HULL False
P (0, 0, 3)
P_ (4, 0, 0)
VOLUME 0.0
FACE NO HULL True
200.0
0.0
PRÓXIMO PONTO (0, 0, 3)
Nova face: {[(0, 2, 0) -> (4, 0, 0)], [(4, 0, 0) -> (0, 0, 3)], [(0, 0, 3) -> (0, 2, 0)]}
Aresta no Not Processed [(0, 0, 3) -> (0, 2, 0)]
Aresta JÁ PROCESSADA
Aresta no Not Processed [(4, 0, 0) -> (0, 0, 3)]
Aresta JÁ PROCESSADA
Aresta no Not Processed [(0, 2, 0) -> (4, 0, 0)]
Aresta JÁ PROCESSADA
Aresta no Not Processed [(0, 0, 3) -> (0, 2, 0

In [60]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [61]:
print_faces(H, P)

f 2 5 1
f 1 2 5
f 5 1 6
f 6 5 2
f 2 6 5
f 1 6 4
f 4 1 3
f 3 4 1
f 1 3 2
f 3 2 5
f 5 3 1
f 6 4 3
f 3 6 4
